In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cifar10
import cifar10_input
import time
import cv2

batch_size = 128
max_steps = 2000

# read dataset
# # tf.app.flags.DEFINE_string('f', '', 'kernel')
# cifar10.maybe_download_and_extract()
# batch_dir = './cifar-10-batches-bin'
# images_train, labels_train = cifar10_input.distorted_inputs(data_dir=batch_dir, batch_size=batch_size)

imgs_train = []
for img_num in range(128):
    imgs_train.append(cv2.resize(cv2.imread("./cats_and_dogs_filtered/train/cats/cat.%d.jpg" % img_num),(24,24)))

labels_train = ["cat"]*128
                    


image_input = tf.placeholder(tf.float32, [None, 24, 24, 3])
text_batch = tf.placeholder('string', shape=[None], name='text_input') 

# define FC network for mapping the output of elmo
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True) #getting ELMo embeddings
elmo_embds = elmo(text_batch, signature="default", as_dict=True)
w_embd = tf.identity(elmo_embds['word_emb'], name='elmo_word_embd') #?xTxD 
class_embd = w_embd[:,0,:] #?xD

w1 = tf.Variable(tf.truncated_normal((512, 400), dtype=tf.float32))
b1 = tf.Variable(np.random.rand(400), dtype=tf.float32)
w2 = tf.Variable(np.random.rand(400, 300), dtype=tf.float32)
b2 = tf.Variable(np.random.rand(300), dtype=tf.float32)
                 
output_text = tf.add(tf.matmul(class_embd, w1), b1)
output_text = tf.nn.relu(output_text)
output_text = tf.add(tf.matmul(output_text, w2), b2)
output_text = tf.nn.relu(output_text)


# define image model
def generate_weight(shape, stddev, w):
    weight = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if w is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(weight), w, name='weight_loss')
        tf.add_to_collection('losses',weight_loss)
    return weight

def generate_bias(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

def conv_2d(x, W):
    return tf.nn.conv2d(x,W, strides=[1,1,1,1], padding="SAME")

def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")



# first convolutional layer
weight1 = generate_weight(shape=[3, 3, 3, 64],stddev=5e-2, w=0.0)
output1 = conv_2d(image_input, weight1)
bias1 = generate_bias([64])
output1 = tf.nn.relu(tf.add(output1, bias1))
output1 = max_pool(output1)
output1 = tf.layers.batch_normalization(output1, training=True)

#second convolutional layer
weight2 = generate_weight(shape=[3, 3, 64, 128],stddev=5e-2, w=0.0)
output2 = conv_2d(output1, weight2)
bias2 = generate_bias([128])
output2 = tf.nn.relu(tf.add(output2, bias2))
output2 = max_pool(output2)
output2 = tf.layers.batch_normalization(output2, training=True)

#third convolutional layer
weight3 = generate_weight(shape=[3, 3, 128, 300],stddev=5e-2, w=0.0)
output3 = conv_2d(output2, weight3)
bias3 = generate_bias([300])
output3 = tf.nn.relu(tf.add(output3, bias3))
output3 = max_pool(output3)
output3 = tf.layers.batch_normalization(output3, training=True)

# implement global pool
output4 = tf.reduce_mean(output3, axis=[1,2])

# loss function
def loss(image_output, text_output):
    loss_value = tf.losses.mean_squared_error(image_output, text_output)
    return tf.add_n(tf.get_collection('losses'), name = 'total_loss')

loss_value = loss(output4, output_text)
train_opt = tf.train.AdamOptimizer(1e-3).minimize(loss_value)

INFO:tensorflow:Using /var/folders/zy/rlk_3zrx72s_z3zjxx3gsv600000gn/T/tfhub_modules to cache modules.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [2]:
# training

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(max_steps):
        start_time = time.time()
        
        free, loss_value1 = sess.run([train_opt, loss_value], feed_dict = {image_input: imgs_train, text_batch: labels_train})
        duration = time.time() - start_time 
        if step % 10 == 0:
            example_per_sec = batch_size / duration
            sec_per_batch = float(duration) 
            format_str = ('step %d, loss=%.2f(%.1f exaples/sec; %.3f sec/batch)')
            print(format_str % (step, loss_value1, example_per_sec, sec_per_batch))
    
    save_path = saver.save(sess, "./model/model.ckpt")
    print("Model saved in path: %s" % save_path)

step 0, loss=701236.69(290.9 exaples/sec; 0.440 sec/batch)
step 10, loss=410139.25(566.4 exaples/sec; 0.226 sec/batch)
step 20, loss=235201.66(543.6 exaples/sec; 0.235 sec/batch)
step 30, loss=135001.94(515.9 exaples/sec; 0.248 sec/batch)
step 40, loss=81250.17(511.7 exaples/sec; 0.250 sec/batch)
step 50, loss=53339.01(495.9 exaples/sec; 0.258 sec/batch)
step 60, loss=36527.00(522.0 exaples/sec; 0.245 sec/batch)
step 70, loss=25995.29(503.3 exaples/sec; 0.254 sec/batch)
step 80, loss=19236.03(501.8 exaples/sec; 0.255 sec/batch)
step 90, loss=14454.28(496.5 exaples/sec; 0.258 sec/batch)
step 100, loss=10997.41(506.3 exaples/sec; 0.253 sec/batch)
step 110, loss=8546.69(491.6 exaples/sec; 0.260 sec/batch)
step 120, loss=6772.17(492.1 exaples/sec; 0.260 sec/batch)
step 130, loss=5474.82(500.8 exaples/sec; 0.256 sec/batch)
step 140, loss=4478.97(469.7 exaples/sec; 0.273 sec/batch)
step 150, loss=3691.07(510.0 exaples/sec; 0.251 sec/batch)
step 160, loss=3073.75(492.3 exaples/sec; 0.260 sec/

step 1430, loss=1.66(502.4 exaples/sec; 0.255 sec/batch)
step 1440, loss=1.59(472.4 exaples/sec; 0.271 sec/batch)
step 1450, loss=1.52(497.8 exaples/sec; 0.257 sec/batch)
step 1460, loss=1.45(485.3 exaples/sec; 0.264 sec/batch)
step 1470, loss=1.39(486.7 exaples/sec; 0.263 sec/batch)
step 1480, loss=1.33(454.2 exaples/sec; 0.282 sec/batch)
step 1490, loss=1.28(498.4 exaples/sec; 0.257 sec/batch)
step 1500, loss=1.24(493.9 exaples/sec; 0.259 sec/batch)
step 1510, loss=1.21(492.4 exaples/sec; 0.260 sec/batch)
step 1520, loss=1.17(446.7 exaples/sec; 0.287 sec/batch)
step 1530, loss=1.14(462.8 exaples/sec; 0.277 sec/batch)
step 1540, loss=1.10(488.7 exaples/sec; 0.262 sec/batch)
step 1550, loss=1.07(502.7 exaples/sec; 0.255 sec/batch)
step 1560, loss=1.04(501.2 exaples/sec; 0.255 sec/batch)
step 1570, loss=1.01(494.7 exaples/sec; 0.259 sec/batch)
step 1580, loss=0.98(475.7 exaples/sec; 0.269 sec/batch)
step 1590, loss=0.95(475.9 exaples/sec; 0.269 sec/batch)
step 1600, loss=0.92(486.2 exap

In [3]:
test_cat_img = cv2.resize(cv2.imread("./cats_and_dogs_filtered/train/cats/cat.129.jpg"), (24,24))
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "./model/model.ckpt")
#     sess.run(tf.global_variables_initializer())
    a = sess.run(loss_value, feed_dict = {text_batch: ["cat"], image_input: [test_cat_img]})
    b = sess.run(loss_value, feed_dict = {text_batch: ["dog"], image_input: [test_cat_img]})
    print("The loss of cat image and label 'cat' is: %f" % a)
    print("The loss of cat image and label 'dog' is: %f" % b)

INFO:tensorflow:Restoring parameters from ./model/model.ckpt
The loss of cat image and label 'cat' is: 0.265908
The loss of cat image and label 'dog' is: 326622.843750
